In [1]:
import re

In [2]:
%autosave 0

Autosave disabled


# Python for Genomics 
### by Guilherme Matos Passarini, phD

## Exercise 1: Calculate Recombination Frequency and Estimate Gene Distances
### Write a program that accepts inputs and estimated gene distances based on the frequency of the Recombinant and Non-Recombinant Genotypes.

In [3]:
AB = int(input('Please enter total count of observed AB, non-recombinants: '))
ab = int(input('Please enter total count of observed ab, non-recombinants: '))
aB = int(input('Please enter total count of observed aB, recombinants: '))
Ab = int(input('Please enter total count of observed Ab, recombinants: '))

Please enter total count of observed AB, non-recombinants: 121
Please enter total count of observed ab, non-recombinants: 200
Please enter total count of observed aB, recombinants: 23
Please enter total count of observed Ab, recombinants: 21


In [4]:
freq = (Ab + aB) / (AB + ab + Ab + aB)
print(f'Estimated Recombinant Frequency: {freq:.4f}')

Estimated Recombinant Frequency: 0.1205


In [5]:
distance = (AB + ab + Ab + aB) / (Ab + aB)
print(f'Estimated Gene Distances: {distance:.2f} bases / kB / MB')

Estimated Gene Distances: 8.30 bases / kB / MB


## Exercise 2: Calculate Allele Frequencies
### Write a program that accepts Genotype Counts and returns the Frequencies of the two Alleles.

In [4]:
pp = int(input('Please enter total count of observed pp genotypes: '))
pq = int(input('Please enter total count of observed pq genotypes: '))
qq = int(input('Please enter total count of observed qq genotypes: '))
n = pp+pq+qq

Please enter total count of observed pp genotypes: 13
Please enter total count of observed pq genotypes: 80
Please enter total count of observed qq genotypes: 30


In [5]:
print(f'Total observed genotypes: {n}')

Total observed genotypes: 123


In [6]:
p_freq = ((2*pp)+pq) / (2*n)
print(f'Frequency of Allele P: {p_freq:.4f}')

Frequency of Allele P: 0.4309


In [7]:
q_freq = ((2*qq)+pq) / (2*n)
print(f'Frequency of Allele Q: {q_freq:.4f}')

Frequency of Allele Q: 0.5691


In [8]:
p_freq + q_freq

1.0

## To summarize:

In [9]:
print(f'Frequency of Allele P: {p_freq:.4f}')
print(f'Frequency of Allele Q: {q_freq:.4f}')
print(f'Total: {(p_freq + q_freq):.1f}')

Frequency of Allele P: 0.4309
Frequency of Allele Q: 0.5691
Total: 1.0


## Exercise 3: 
### Write a script that takes the RNA string GGCCCGAUGGGAGGAGGGCCCUUGGGAUGAUUUGACCGA and:
    - Returns the position where the START and STOP codons begin AUG (start) and UGA/UAG/UAA (stop)
    - Extracts the Open Reading Frame (ORF)/ Exon of the RNA Sequence

#### Begin with a representative sample:

In [41]:
rna = 'GGCCCGAUGGGAGGAGGGCCCUUGGGAUGAUUUGACCGA'

#### In the instructional video, Passarini "brute forces" the code and specifies the 'AUG' and 'UGA' codons because we can clearly see the 'AUG' start codon at index 6 and the 'UGA' terminate codon at index 27.

In [89]:
start = rna.find('AUG')
termination = rna.find('UGA')

print(f'The positions of the start and termination codons are {start} and {termination}, respectively')

The positions of the start and termination codons are 6 and 27, respectively


#### The Open Reading Frame can easily be extracted when we have the start and terminate indices.

In [90]:
orf = rna[start:termination+3]

In [91]:
print(f'ORF: {orf}')

ORF: AUGGGAGGAGGGCCCUUGGGAUGA


### This approach seems to work with a very small codon string...

- How do we let the computer do the work when we have a very large codon input and it is not possible to "clearly see" the termination codons?
- Given the three possible termination codons of "UGA", "UAG", and "UAA", we need a more sophisticated approach.
- Given the likelihood that we will be dealing with a significantly larger string of codons, we need a better method.

### Let's do this programmatically so that we can apply this method to a much larger string of codons

In [42]:
exon_start_list = [x.start() for x in re.finditer('AUG',rna)]

#### Here, we generate a list of indices where 'AUG' is found:  index 6, and index 26 (N.B. the first index is 0)

In [43]:
exon_start_list

[6, 26]

#### Upon closer inspection, we have a few issues:
- The 'AUG' at index 26 does not have a corresponding termination codon.
- The 'UGA' at index 27 IS the termination of the ORF which begins at index 6, but because it is preceeded by 'A', it is being recognized as 'AUG' at index 26.
- This instance cannot be both a start and terminate codon.

#### Let's refine our definition of a start start to include the fact that it begins at an index which is a MULTIPLE of 3:

In [54]:
exon_start_list = [x.start() for x in re.finditer('AUG',rna) if (x.start() % 3) == 0]

In [45]:
exon_start_list

[6]

#### Now, it is correctly identifying those start codons which are not simply 'AUG', but those that have some positional requirements.
#### We can now define a function called 'find_stop', which will identify the termination codon positions.

In [46]:
def find_stop(exon_start_list):
    '''
    Accepts a list of start codon position indeces and outputs a list of corresponding termination
    codon position indeces.
    '''
    stop_list = []
    for i in (exon_start_list):
        if rna.find('UGA',i):
            pos = rna.find('UGA',i)
        elif rna.find('UAG',i):
            pos = rna.find('UAG',i)
        elif rna.find('UAA',i):
            pos = rna.find('UAA',i)
        else:
            pos =  0
        stop_list.append(pos)
    return stop_list

#### Properly, the terminate codon that correlates with the identified start codon at index 6 can be found at index 27.

In [47]:
find_stop(exon_start_list)

[27]

## Now test it for real on a larger codon string:

In [51]:
rna = 'GGCCCGAUGGGAGGAGGGCCCUUGGGAUGAUUUGACCGAGGCCCGAUGGGAGGAGGGCCCUUGGGAUGAUUUGACCGAGGCCCGAUGGGAGGAGGGCCCUUGGGAUGAUUUGACCGAGGCCCGAUGGGAGGAGGGCCCUUGGGAUGAUUUGACCGAGGCCCGAUGGGAGGAGGGCCCUUGGGAUGAUUUGACCGAGGCCCGAUGGGAGGAGGGCCCUUGGGAUGAUUUGACCGA'

In [52]:
rna

'GGCCCGAUGGGAGGAGGGCCCUUGGGAUGAUUUGACCGAGGCCCGAUGGGAGGAGGGCCCUUGGGAUGAUUUGACCGAGGCCCGAUGGGAGGAGGGCCCUUGGGAUGAUUUGACCGAGGCCCGAUGGGAGGAGGGCCCUUGGGAUGAUUUGACCGAGGCCCGAUGGGAGGAGGGCCCUUGGGAUGAUUUGACCGAGGCCCGAUGGGAGGAGGGCCCUUGGGAUGAUUUGACCGA'

### Just by copying the same codon string 6 times, we now have a list of 6 indices where 'AUG' can be found:

In [55]:
exon_start_list

[6, 45, 84, 123, 162, 201]

### and 6 indices where 'UGA',' UAG', or 'UAA' can be found:

In [63]:
exon_term_list = find_stop(exon_start_list)

In [64]:
exon_term_list

[27, 66, 105, 144, 183, 222]

### Let us extract each of the ORF's

In [78]:
orfs = []
for (i,j) in zip(exon_start_list,exon_term_list):
    orf = rna[i:j+3]
    orfs.append(orf)

In [79]:
orfs

['AUGGGAGGAGGGCCCUUGGGAUGA',
 'AUGGGAGGAGGGCCCUUGGGAUGA',
 'AUGGGAGGAGGGCCCUUGGGAUGA',
 'AUGGGAGGAGGGCCCUUGGGAUGA',
 'AUGGGAGGAGGGCCCUUGGGAUGA',
 'AUGGGAGGAGGGCCCUUGGGAUGA']

## Exercise 4: 
### Given a DNA sequence in lowercase letters:
    - Convert to UPPERCASE
    - Slice the string from the 5th character to the 10th
    - Transcription: convert the DNA string into an RNA character string